In [1]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ibmiotf.application
from Queue import Queue
%matplotlib inline

Using TensorFlow backend.


In [ ]:
!pip install h5py

In [ ]:
#!pip install keras
#!pip install ibmiotf

In [ ]:
!rm watsoniotp.*
!wget http://www.romeokienzler.com/watsoniotp.healthy.pickle
!wget http://www.romeokienzler.com/watsoniotp.broken.pickle

In [2]:
data_healthy = pickle.load(open('watsoniotp.healthy.pickle', 'rb'))
data_broken = pickle.load(open('watsoniotp.broken.pickle', 'rb'))

In [3]:
data_healthy = data_healthy.reshape(3000,3)
data_broken = data_broken.reshape(3000,3)

In [4]:
def scaleData(data):
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [5]:
data_healthy_scaled = scaleData(data_healthy)
data_broken_scaled = scaleData(data_broken)

In [6]:
timesteps = 10
dim = 3
samples = 3000
data_healthy_scaled_reshaped = data_healthy_scaled
#reshape to (300,10,3)
data_healthy_scaled_reshaped.shape = (samples/timesteps,timesteps,dim)

In [7]:
losses = []

def handleLoss(loss):
        global losses
        losses+=[loss]
        print loss

In [8]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        handleLoss(logs.get('loss'))

In [9]:
# design network

model = Sequential()
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(Dense(3))
model.compile(loss='mae', optimizer='adam')

def train(data):
    data.shape = (300, 10, 3)
    model.fit(data, data, epochs=50, batch_size=72, validation_data=(data, data), verbose=0, shuffle=False,callbacks=[LossHistory()])
    data.shape = (3000, 3)

def score(data):
    data.shape = (300, 10, 3)
    yhat =  model.predict(data)
    yhat.shape = (3000, 3)
    return yhat

In [10]:
for i in range(20):
    
    print "----------------"
    train(data_healthy_scaled)
    yhat_healthy = score(data_healthy_scaled)
    yhat_broken = score(data_broken_scaled)
    data_healthy_scaled.shape = (3000, 3)
    data_broken_scaled.shape = (3000, 3)

----------------
0.469564
0.490879
0.360382
0.487857
0.345799
0.440619
0.459061
0.321957
0.443454
0.292343
0.379238
0.389259
0.235099
0.344082
0.210462
0.285035
0.30329
0.242922
0.300793
0.233781
0.27502
0.286969
0.194904
0.282287
0.181121
0.262627
0.289238
0.168059
0.284638
0.167197
0.256431
0.280671
0.159573
0.268897
0.162121
0.242561
0.263569
0.161325
0.253873
0.16494
0.234426
0.253681
0.160437
0.24622
0.160218
0.22873
0.249197
0.150907
0.243
0.151611
0.225086
0.247593
0.142562
0.240886
0.145399
0.221926
0.245017
0.137459
0.236674
0.141483
0.217979
0.240678
0.135224
0.231528
0.139369
0.21418
0.236337
0.133201
0.227255
0.137206
0.211013
0.233157
0.129633
0.224364
0.134428
0.208546
0.231089
0.125699
0.222332
0.131401
0.206534
0.229417
0.122528
0.220267
0.128619
0.20467
0.227953
0.119998
0.218241
0.126365
0.203016
0.226505
0.118112
0.216369
0.124941
0.201482
0.224899
0.117083
0.21449
0.12454
0.199942
0.222841
0.117261
0.212184
0.125262
0.198279
0.220436
0.118177
0.209843
0.126266
0.196

In [11]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
